In [ ]:
# Importing all the necessary libraries
import numpy as np
import pandas as pd
import random
import os, cv2
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D

In [ ]:
# acquire MNIST data
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# reshape data for an MLP input
import numpy as np

train_images = np.reshape(train_images, (-1, 784))
test_images = np.reshape(test_images, (-1, 784))

# normalize data
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
print(train_images.shape)
print(test_images.shape)
print(train_labels.shape)
print(test_labels.shape)

(60000, 784)
(10000, 784)
(60000, 10)
(10000, 10)


In [ ]:
def create_clients(train_images, train_labels, num_clients=10, initial='clients'):
    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(train_images, train_labels))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))} 

In [ ]:
clients = create_clients(train_images, train_labels, num_clients=10, initial='client')

In [ ]:
def batch_data(data_shard, bs=32):
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [ ]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(len(test_labels))

In [ ]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(784,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(10))
        model.add(Activation("softmax"))
        return model   

In [ ]:
lr = 0.01 
comms_round = 100
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               )          

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

In [ ]:
def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(test_images, test_labels,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(test_images)
    loss = cce(test_labels, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(test_labels, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [ ]:
def fl_model(clients_batched, test_batched, comms_round, loss, optimizer, metrics):
    #initialize global model
    smlp_global = SimpleMLP()
    global_model = smlp_global.build(784,10)

    #commence global training loop
    for comm_round in range(comms_round):
            
        # get the global model's weights - will serve as the initial weights for all local models
        global_weights = global_model.get_weights()
    
        #initial list to collect local model weights after scalling
        scaled_local_weight_list = list()

        #randomize client data - using keys
        client_names= list(clients_batched.keys())
        random.shuffle(client_names)
    
        #loop through each client and create new local model
        for client in client_names:
            smlp_local = SimpleMLP()
            local_model = smlp_local.build(784, 10)
            local_model.compile(loss=loss, 
                          optimizer=optimizer, 
                          metrics=metrics)
        
            #set local model weight to the weight of the global model
            local_model.set_weights(global_weights)
        
            #fit local model with client's data
            local_model.fit(clients_batched[client], epochs=1, verbose=0)
        
            #scale the model weights and add to list
            scaling_factor = weight_scalling_factor(clients_batched, client)
            scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
            scaled_local_weight_list.append(scaled_weights)
        
            #clear session to free memory after each communication round
            K.clear_session()
        
        #to get the average over all the local model, we simply take the sum of the scaled weights
        average_weights = sum_scaled_weights(scaled_local_weight_list)
    
        #update global model 
        global_model.set_weights(average_weights)

        #test global model and print out metrics after each communications round
        for(test_images, test_labels) in test_batched:
            global_acc, global_loss = test_model(test_images, test_labels, global_model, comm_round)
            
            # Make predictions on the test set
            y_pred = global_model.predict(test_images)
            y_pred = np.argmax(y_pred, axis=1)
            
            # Get the true classes
            true_classes = np.argmax(test_labels, axis = 1)
      
             # Print the classification report
            report = classification_report(true_classes, y_pred)
            from sklearn.metrics import confusion_matrix
            confusion_matrix = confusion_matrix(true_classes, y_pred)
            if comm_round == 99:
              print(confusion_matrix)
              print(report)

In [ ]:
fl_model(clients_batched, test_batched, comms_round, loss, optimizer, metrics)

313/313 [==============================] - 0s 1ms/step
comm_round: 0 | global_acc: 90.820% | global_loss: 1.6192830801010132
313/313 [==============================] - 0s 1ms/step
comm_round: 1 | global_acc: 92.660% | global_loss: 1.586478352546692
313/313 [==============================] - 0s 1ms/step
comm_round: 2 | global_acc: 93.480% | global_loss: 1.5689648389816284
313/313 [==============================] - 0s 1ms/step
comm_round: 3 | global_acc: 94.040% | global_loss: 1.5590898990631104
313/313 [==============================] - 0s 1ms/step
comm_round: 4 | global_acc: 94.560% | global_loss: 1.5523031949996948
313/313 [==============================] - 0s 1ms/step
comm_round: 5 | global_acc: 94.760% | global_loss: 1.5463365316390991
313/313 [==============================] - 0s 1ms/step
comm_round: 6 | global_acc: 95.150% | global_loss: 1.542054295539856
313/313 [==============================] - 0s 1ms/step
comm_round: 7 | global_acc: 95.320% | global_loss: 1.5389293432235718
31

In [ ]:
def test_model(test_images, test_labels,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(test_images)
    loss = cce(test_labels, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(test_labels, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [ ]:
SGD_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(len(train_labels)).batch(320)
smlp_SGD = SimpleMLP()
SGD_model = smlp_SGD.build(784, 10) 

SGD_model.compile(loss=loss, 
              optimizer=optimizer, 
              metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

#test the SGD global model and print out metrics
for(test_images, test_labels) in test_batched:
        SGD_acc, SGD_loss = test_model(test_images, test_labels, SGD_model, 1)

        # Make predictions on the test set
        y_pred = SGD_model.predict(test_images)
        y_pred = np.argmax(y_pred, axis=1)
            
        # Get the true classes
        true_classes = np.argmax(test_labels, axis = 1)
      
        # Print the classification report and the confusion matrix
        report = classification_report(true_classes, y_pred)
        from sklearn.metrics import confusion_matrix
        confusion_matrix = confusion_matrix(true_classes, y_pred)
        print(confusion_matrix)
        print(report)

313/313 [==============================] - 0s 1ms/step
comm_round: 1 | global_acc: 95.060% | global_loss: 1.5425121784210205
313/313 [==============================] - 0s 1ms/step
[[ 966    0    1    2    0    4    5    1    1    0]
 [   0 1116    3    2    0    1    3    2    8    0]
 [   6    1  974    8    7    1    9    7   15    4]
 [   0    1   13  959    0   13    0    9   10    5]
 [   1    1    4    1  932    1   10    3    3   26]
 [  10    1    0   18    4  831    9    1   10    8]
 [  10    3    5    0    8   11  918    1    2    0]
 [   2    8   23    4    3    1    0  964    3   20]
 [   6    2    5   16    8    8   10    9  904    6]
 [   9    7    1   10   20    3    1    9    7  942]]
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       980
           1       0.98      0.98      0.98      1135
           2       0.95      0.94      0.95      1032
           3       0.94      0.95      0.94      1010
           4      

In [ ]:
# Label-flipping

In [ ]:
## Pseudocode for flipping perecent p of labels.

# Store all possible labels in list all_labels.
# k =  (total samples of client * p)/100     

# For one particular client dataset- do following.
# Get k random indexes of dataset and store in a list all_indexes (list contains k random numbers from 0 - number of samples for that client).
# For each value in the all_indexes list obtained above - do following
# Replace label with some random label from list all_labels

In [ ]:
client_1,label_1=zip(*clients['client_1'])

In [ ]:
# before replacing labels for label_1
for i, label in enumerate(label_1):
    print(i, label)

Streaming output truncated to the last 5000 lines.
1000 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
1001 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
1002 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
1003 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
1004 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
1005 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
1006 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
1007 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
1008 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
1009 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
1010 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
1011 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
1012 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
1013 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
1014 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
1015 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
1016 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
1017 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
1018 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
1019 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
1020 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
1021 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
1022 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
1023 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
1024 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
1025 [0. 0. 0. 0. 0. 1. 

In [ ]:
# getting all labels under all_labels variable
all_labels = ()
clients_1 = ['client_1','client_2','client_3','client_4','client_5','client_6','client_7','client_8','client_9','client_10']
for client in range(len(clients_1)):
    name = clients_1[client]
    image,label = zip(*clients[name])
    all_labels=all_labels+(label)
# print(list(all_labels))


In [ ]:
len(all_labels)

60000

In [ ]:
# finding the k
p=20
k= len(clients['client_1'])*p//100
print(k)

1200


In [ ]:
# unzipped the client_1 into images and labels
len_of_one_client_dataset = len(client_1)
print(len_of_one_client_dataset)

6000


In [ ]:
all_indexes = np.random.randint(0, 6000, k)

print(all_indexes)

[2149 3229  275 ... 5048 4025  940]


In [ ]:
label_1 = list(label_1)

# Replace the label at each index in all_indexes with a random label from all_labels
for index in all_indexes:
    label_1[index] = random.choice(all_labels)

In [ ]:
print(label_1)

[array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32), array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32), array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32), array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32), array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 1., 0., 0., 0., 

In [ ]:
# cross checking from all_labels
all_labels[2149]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [ ]:
type(label_1)

list

In [ ]:
# assigning the replaced labels to particular client
# Replace the labels in the clients dictionary
clients["client_1"] = list(zip(client_1, label_1))

In [ ]:
# cross checking if it is assigned or not
clients['client_1'][2149]

(array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [ ]:
# cross checking after replacing for label_1
for i in all_indexes:
    print(label_1[i])

[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0

In [ ]:
fl_model(clients_batched, test_batched, comms_round, loss, optimizer, metrics)

313/313 [==============================] - 0s 1ms/step
comm_round: 0 | global_acc: 12.840% | global_loss: 2.2994704246520996
313/313 [==============================] - 0s 1ms/step
comm_round: 1 | global_acc: 18.100% | global_loss: 2.2944114208221436
313/313 [==============================] - 0s 1ms/step
comm_round: 2 | global_acc: 25.580% | global_loss: 2.2890374660491943
313/313 [==============================] - 0s 1ms/step
comm_round: 3 | global_acc: 35.450% | global_loss: 2.2832844257354736
313/313 [==============================] - 0s 1ms/step
comm_round: 4 | global_acc: 44.790% | global_loss: 2.277066707611084
313/313 [==============================] - 0s 1ms/step
comm_round: 5 | global_acc: 52.500% | global_loss: 2.2703030109405518
313/313 [==============================] - 0s 1ms/step
comm_round: 6 | global_acc: 57.470% | global_loss: 2.2629313468933105
313/313 [==============================] - 0s 1ms/step
comm_round: 7 | global_acc: 61.330% | global_loss: 2.254866361618042
31

In [ ]:
SGD_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(len(train_labels)).batch(320)
smlp_SGD = SimpleMLP()
SGD_model = smlp_SGD.build(784, 10) 

SGD_model.compile(loss=loss, 
              optimizer=optimizer, 
              metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

#test the SGD global model and print out metrics
for(test_images, test_labels) in test_batched:
        SGD_acc, SGD_loss = test_model(test_images, test_labels, SGD_model, 1)

        # Make predictions on the test set
        y_pred = SGD_model.predict(test_images)
        y_pred = np.argmax(y_pred, axis=1)
            
        # Get the true classes
        true_classes = np.argmax(test_labels, axis = 1)
      
        # Print the classification report and the confusion matrix
        report = classification_report(true_classes, y_pred)
        from sklearn.metrics import confusion_matrix
        confusion_matrix = confusion_matrix(true_classes, y_pred)
        print(confusion_matrix)
        print(report)

313/313 [==============================] - 0s 1ms/step
comm_round: 1 | global_acc: 89.050% | global_loss: 1.6824482679367065
313/313 [==============================] - 0s 1ms/step
[[ 955    0    4    1    0    5    8    1    6    0]
 [   0 1105    4    3    1    0    4    1   17    0]
 [  16   13  880   20   19    1   21   16   43    3]
 [   3    2   20  901    0   34    5   20   20    5]
 [   1    6    3    0  890    2   19    1    6   54]
 [  21    5    4   53   23  700   25   13   38   10]
 [  17    3   11    1   12   19  893    0    2    0]
 [   5   23   30    3   10    0    0  904    5   48]
 [   9   13   14   36   12   32   18   10  810   20]
 [  10    8    8   11   52   13    2   30    8  867]]
              precision    recall  f1-score   support

           0       0.92      0.97      0.95       980
           1       0.94      0.97      0.96      1135
           2       0.90      0.85      0.88      1032
           3       0.88      0.89      0.88      1010
           4      